**Introduction**

The ultimate goal is to detect fish point patterns for unique identification via topological methods.
There are some papers on it that it is possible to distinguish certain types of fish with their respective dot patters,
but it remains for example unclear for how many fishes their methods can actually work, since lab envirnonments have at maximum 30 fishes.
At the same time, a meaningful encoding is difficult to achieve.
We therefore try to use persistent homology methods to "save" the dot patterns in a machine learning friendly way, which then can be used to identify different fishes.
As a first step, we want to use PersLay to find out how many different classes of random pattern can be distinguished at all with certain architectures.

For that purpose, we create an artificial dataset of 2d points.
We fix the number of classes, and the number of "variations" of that point-image (noise added) to account for variations
in previous image processing methods.

In [18]:

def create_test_dataset(number_of_different_classes=100, images_per_class=10, dimensions=2, number_of_points=10,
                        noise=0.1, overwrite=False):
    """Create random points in a 2D-grid from -1 to 1
    :param number_of_different_classes: THe number of wished classes
    :type number_of_different_classes: int
    :param images_per_class: The number of noised images that one class should contain
    :type images_per_class: int
    :param dimensions: dimension of the dataset
    :type dimensions: int
    :param number_of_points: Number of datapoints for each image
    :type number_of_points: int
    :param noise: The added noise to the data
    :type noise: float
    :param overwrite: Whether a new data file will be written
    :type overwrite: bool
    :return: None
    :rtype: None
    """
    if overwrite or not pathlib.Path('project_data_training.pkl').exists():
        np.random.seed(0)
        json_file = dict()
        json_file["info"] = "Dataset for random point patterns"
        data = dict()
        noised_images = dict()
        counter = 0
        # Create the classes
        for i in range(number_of_different_classes):
            samples = np.array(np.random.random(number_of_points * dimensions) * 20 - 10)
            reshaped_samples = np.reshape(samples, (number_of_points, dimensions))
            # Create the noised images
            for j in range(images_per_class):
                noised_images[counter] = dict()
                sample_noise = np.array(np.random.random(number_of_points * dimensions) * noise - (.5 * noise))
                reshaped_sample_noise = np.reshape(sample_noise, (number_of_points, dimensions))
                noised_images[counter]["image_id"] = counter
                noised_images[counter]["class"] = i
                noised_images[counter]["image"] = reshaped_samples + reshaped_sample_noise
                counter += 1
            data["noised_images"] = noised_images
        json_file["data"] = data
        # Only when debugging
        # print(json_file)
        with open('project_data_training.pkl', 'wb') as f:
            pickle.dump(json_file, f)
        print("New dataset created")

Furthermore, we need a method to calculate the persistence diagrams.
Here we can see that our appraoch seems meaningful,
since we already get very different persistence diagrams for different classes,
but similar ones within the variation group. To turn on drawing, set the "draw" variable to true in the run.

In [19]:

def write_dataset_of_persistence_diagrams(max_simplices_dim=2, overwrite=False, draw=False,
                                          filter_by_immediate_deaths=False, filter_by_max_value=False,
                                          filter_to_one_dimensional=False):
    """Saves a dataset of persistence diagrams based on the fed data.
    :param max_simplices_dim: Max. dimension of simplicies
    :type max_simplices_dim: int
    :param overwrite: Whether a new dataset of persistence diagrams is created
    :type overwrite: bool
    :param draw: Whether plots for EACH persistence diagram is being showed
    :type draw: bool
    :param filter_by_immediate_deaths: Whether to filter out persistence pairs on the diagonal
    :type filter_by_immediate_deaths: bool
    :param filter_by_max_value: Whether to filter out persistence pairs with infinity as death values
    :type filter_by_max_value: bool
    :param filter_to_one_dimensional: Whether to filter out persistence pairs which do not have the dimension 1.
    :type filter_to_one_dimensional: bool
    :return: None
    :rtype: None
    """
    if not pathlib.Path('project_data_training.pkl').exists():
        raise FileNotFoundError("We cannot find the project data file!")
    if overwrite or not pathlib.Path('pers_diagram_training.pkl').exists():
        # Load data
        with open("project_data_training.pkl", 'rb') as f:
            data = pickle.load(f)
        persistence_barcodes = dict()
        current_class = -1
        current_variation = 0
        for key_id, noised_image in data["data"]["noised_images"].items():

            persistence_barcode = dict()
            training_data = noised_image["image"]
            persistence_barcode["class"] = noised_image["class"]
            persistence_barcode["barcode_id"] = noised_image["image_id"]
            # Saving class and variation for display
            if current_class == persistence_barcode['class']:
                current_variation = current_variation +1
            else:
                current_class = persistence_barcode["class"]
                current_variation = 0
            # Calculate Filtered Complex on the training data
            filtered_complex = FilteredComplexes(training_data)
            bounday_matrix, dimensional_array, distance_list = filtered_complex.fit(max_simplices_dim=max_simplices_dim,
                                                                                    k_nearest_neighours=5)
            matrix_reductor = MatrixReduction(bounday_matrix, dimensional_array, distance_list=distance_list)
            matrix_reductor.reduce_boundary_matrix()
            if draw:
                matrix_reductor.draw_persistence_diagram(plot_inf=False, title=f"Persistence diagram of class {persistence_barcode['class']}, variation {current_variation}")

            barcode = matrix_reductor.get_barcode_of_reduced_matrix()
            barcode[barcode == np.inf] = -1
            max_value = np.max(barcode["death_value"]) + 100
            barcode[barcode == -1] = max_value
            if filter_by_max_value:
                barcode = barcode[barcode["death_value"] != max_value]
                print(f"Barcode after max filtering: {len(barcode)}")
            if filter_by_immediate_deaths:
                barcode = barcode[barcode["birth_value"] != barcode["death_value"]]
                print(f"Barcode after immediate deaths filtering: {len(barcode)}")
            if filter_to_one_dimensional:
                barcode = barcode[barcode["dimension"] == 1]
                print(f"Barcode after filtering for one dimensional simplices: {len(barcode)}")
            barcode_numpy = barcode[["birth_value", "death_value"]].to_numpy()
            persistence_barcode["barcode"] = barcode_numpy

            persistence_barcodes[key_id] = persistence_barcode

        with open('pers_diagram_training.pkl', 'wb') as f:
            pickle.dump(persistence_barcodes, f)


In order to train on the persistence diagrams, we need to create a custom dataset.

In [20]:
from torch.utils.data import Dataset

class CustomPersistenceDiagramDataset(Dataset):
    """Custom Dataset inherit after the conventions of pytorchin order to use it in a DataLoader."""
    def __init__(self, project_data_file="pers_diagram_training.pkl"):
        with open(project_data_file, 'rb') as f:
            x = pickle.load(f)
            self.data = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return np.array(item["barcode"], dtype=float), item["class"]

As the last step, we train the neural network based on the PersLay on the persistence diagrams.
That one does not work yet though...


In [21]:

def run_project_perslay(rho_output_dim_q=25):
    """Runs the training loop for the perslay neural network. DIUSCLAIMER: DOES NOT WORK! Plots a loss plot.
    :param rho_output_dim_q: Output dimention of the perslay
    :type rho_output_dim_q: int
    :return: None
    :rtype: None
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # for debugging
    # device = 'cpu'
    dataset = CustomPersistenceDiagramDataset()
    data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)
    perslay = PersLay(layer_type="persistence_diagram", rho_output_dim_q=rho_output_dim_q)
    perslay.to(device)
    # Loss function
    criterion = torch.nn.MSELoss()
    criterion.requires_grad = True
    optimizer = optim.SGD(perslay.parameters(), lr=0.1)
    lambda1 = lambda epoch: max(0.99 ** epoch, 0.00001 / 0.1)
    scheduler = LambdaLR(optimizer, lr_lambda=lambda1)
    loss_stats = []
    epoch_stats = []
    i = 0
    for t in range(1000):
        # Training
        for idx, (train_data, train_labels) in enumerate(data_loader):
            optimizer.zero_grad()
            # Transfer to GPU
            train_data = train_data.to(device)
            train_labels = train_labels.to(device)
            # Forward pass: compute predicted y
            pred_prob = perslay.forward(torch.squeeze(train_data, 0).float())
            y_pred = np.argmax(pred_prob.cpu().detach().numpy(), axis=0)

            # Compute and print loss
            loss = criterion(torch.FloatTensor([y_pred]), train_labels.type(torch.FloatTensor))
            loss_stats.append(loss.cpu().detach().numpy())
            epoch_stats.append(i)
            i = i + 1
            if t % 10 == 0:
                print(f"Epoch {t}")
                print(f"Predicted: {y_pred}")
                print(f"Actual: {train_labels.cpu().detach().numpy()}")
                print("loss :", loss.cpu().detach().numpy())
            loss.requires_grad = True
            loss.backward()
            optimizer.step()
        scheduler.step()
    print("Training finished")
    df = pandas.DataFrame(data=np.array([epoch_stats, loss_stats]).transpose(), columns=["step", "loss"])
    fig = px.line(df, x="step", y="loss")
    fig.show()

Now let us run all the code!
The overwrite attribute allows to overwrite the existing saved dataset (we save some time skipping the creation every time).
draw=True will let the persistence diagrams calculated pop up visually.

In [ ]:
import pathlib
import pickle

import numpy as np
import pandas
import plotly.express as px
import torch
from torch import optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

from persistent_homology import FilteredComplexes, \
    MatrixReduction, PersLay
overwrite = True
draw = True
transform = transforms.Compose([
    transforms.ToTensor()
])
classes = 10
max_simplices_dim = 2
create_test_dataset(number_of_different_classes=classes, images_per_class=3, number_of_points=15, noise=0.5,
                    overwrite=overwrite)
write_dataset_of_persistence_diagrams(max_simplices_dim=max_simplices_dim, overwrite=overwrite, draw=draw,
                                      filter_by_immediate_deaths=True, filter_by_max_value=True,
                                      filter_to_one_dimensional=True)

New dataset created
All radii: [ 0.          5.15939238  5.28524743  5.92925472  6.21834547  6.63292745
  6.68670184  6.79202428  7.26493206  7.30768955  7.38248501  7.67295683
  7.84440952  7.84440952  7.91414901  7.95237288  8.43318638  8.44768168
  8.52949001  8.826579    8.86137315  8.87853547  9.02149458  9.08700558
  9.27444436  9.33633399  9.86117849  9.95909629 10.11821695 10.14811464
 10.17981538 10.38387521 10.46600496 10.79087845 10.83710136 10.9191797
 10.97279305 11.34789002 11.75777447 12.00248115 12.04346156 12.41223949
 12.6086717  12.60948469 12.97517012 13.0099183  13.1206103  13.2475455
 13.70351386 14.06077411 14.69176301 14.72823518 15.23142916 15.23587804
 15.70329139 15.74904972 16.59660384 16.71419428 16.9003651  17.00316465
 17.64825537 17.70230087 17.94834356 18.94369669 19.24011284 19.35474261
 19.87910478 20.86743804 21.11447571 22.96159979]
Calculating boundary_matrix: This may take a while...


In [ ]:
run_project_perslay(rho_output_dim_q=classes)